In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
tf.__version__

'2.9.1'

In [3]:
import tensorflow
tensorflow.keras.__version__

'2.9.0'

In [4]:
def __load_dataframe(is_train=True):
    base_path="./data/facial-keypoints-detection/"
    if is_train:
        file_name = "training.csv"
    else:
        file_name = "test.csv"
    return pd.read_csv(base_path + file_name)

In [5]:
def __convert_image_dataset(raw_image_infos):
    image_infos = []
    for raw_img_info in raw_image_infos:
        image_infos.append(list(map(int, raw_img_info.split())))
    image_infos = np.array(image_infos).reshape(-1,96,96,1)
    return image_infos

In [6]:
def fill_null_data(df, fill_value):
    df = df.fillna(fill_value)
    return df

In [11]:
def get_dataset_xy(is_train=True):
    df = __load_dataframe(is_train)
    y = df.iloc[:,:-1]
    x = df.iloc[:,-1]
    print(y)
    y = fill_null_data(y, -100.0)
    x = __convert_image_dataset(x)
    print(y)
    return x, y.values

In [8]:
def normalize_image(x):
    x = x.astype(np.float32)/255.0
    return x

In [9]:
def split_data(x, y, train_ratio=0.8):
    row = x.shape[0]
    indices = np.random.choice(row, row)
    x = tf.gather(x, indices=indices).numpy()
    y = tf.gather(y, indices=indices).numpy()

    train_count = int(row * train_ratio)
    valid_count = row - train_count
    x0, x1 = tf.split(x, [train_count, valid_count])
    y0, y1 = tf.split(y, [train_count, valid_count])
    x = [x0, x1]
    y = [y0, y1]
    print(x0.shape, x1.shape, y0.shape, y1.shape,)
    return x, y

In [12]:
train_x, train_y = get_dataset_xy(is_train=True)

      left_eye_center_x  left_eye_center_y  right_eye_center_x  \
0             66.033564          39.002274           30.227008   
1             64.332936          34.970077           29.949277   
2             65.057053          34.909642           30.903789   
3             65.225739          37.261774           32.023096   
4             66.725301          39.621261           32.244810   
...                 ...                ...                 ...   
7044          67.402546          31.842551           29.746749   
7045          66.134400          38.365501           30.478626   
7046          66.690732          36.845221           31.666420   
7047          70.965082          39.853666           30.543285   
7048          66.938311          43.424510           31.096059   

      right_eye_center_y  left_eye_inner_corner_x  left_eye_inner_corner_y  \
0              36.421678                59.582075                39.647423   
1              33.448715                58.856170  

In [13]:
train_y

array([[  66.03356391,   39.00227368,   30.22700752, ...,   72.93545865,
          43.13070677,   84.48577444],
       [  64.33293617,   34.9700766 ,   29.9492766 , ...,   70.26655319,
          45.46791489,   85.48017021],
       [  65.05705263,   34.90964211,   30.90378947, ...,   70.19178947,
          47.27494737,   78.65936842],
       ...,
       [  66.69073171,   36.84522146,   31.66641951, ..., -100.        ,
          49.46257171,   78.11712   ],
       [  70.96508235,   39.85366588,   30.54328471, ..., -100.        ,
          50.06518588,   79.58644706],
       [  66.93831111,   43.42450963,   31.09605926, ..., -100.        ,
          45.90048   ,   82.7730963 ]])

In [8]:
train_x = normalize_image(train_x)

In [9]:
train_x2, train_y2 = split_data(train_x, train_y)

(5639, 96, 96, 1) (1410, 96, 96, 1) (5639, 30) (1410, 30)


In [69]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D

import numpy as np

# class Block():
#     def __init__(self, input_size, conv_filters, kernel_size=(3, 3), conv_stride=(1, 1),
#                 pool_size=(2, 2), pool_stride=(2, 2)):
#         self.block = Sequential([
#             Conv2D(input_shape=input_size, filters=conv_filters[0], kernel_size=kernel_size,
#                     strides=conv_stride, padding="same", activation="relu"),
#             Conv2D(filters=conv_filters[1], kernel_size=(3, 3),
#                     strides=conv_stride, padding="same", activation="relu"),
#             MaxPool2D(pool_size=pool_size, strides=pool_stride),
#         ])

class Facial_Kepoints_Detect():
    def __init__(self, input_size, output_size,
                    init_conv_filters, kernel_size=(3, 3), conv_stride=(1, 1),
                    pool_size=(2, 2), pool_stride=(2, 2)):
        last_input_size = input_size
        # for i in range(2):
        #     filters = (init_conv_filters * (2 * i + 1), init_conv_filters * (2 * i + 2))
        #     blocks += [Block(last_input_size, conv_filters=filters)]
        #     last_input_size = list((np.array(last_input_size[:2]) / pool_size[0]).astype("int64")) + [filters[-1]]
        #     print(filters, last_input_size)
#         last_input_size = last_input_size[0]*last_input_size[1] * filters[-1]
        self.layers = Sequential([
                                Conv2D(input_shape=input_size, filters=init_conv_filters, kernel_size=kernel_size,
                                        strides=conv_stride, padding="same", activation="relu"),
                                Conv2D(filters=init_conv_filters*2, kernel_size=kernel_size,
                                        strides=conv_stride, padding="same", activation="relu"),
                                MaxPool2D(pool_size=pool_size, strides=pool_stride),
                                Conv2D(filters=init_conv_filters*3, kernel_size=kernel_size,
                                        strides=conv_stride, padding="same", activation="relu"),
                                Conv2D(filters=init_conv_filters*4, kernel_size=kernel_size,
                                        strides=conv_stride, padding="same", activation="relu"),
                                MaxPool2D(pool_size=pool_size, strides=pool_stride),
                                Flatten(),
                                Dense(24*24*32, activation="relu"),
                                Dense(output_size, activation="relu")
        ])
        self.layers.summary()



In [70]:
model = Facial_Kepoints_Detect(input_size=[96,96,1], output_size=30, init_conv_filters=8,)


Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_56 (Conv2D)          (None, 96, 96, 8)         80        
                                                                 
 conv2d_57 (Conv2D)          (None, 96, 96, 16)        1168      
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 48, 48, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_58 (Conv2D)          (None, 48, 48, 24)        3480      
                                                                 
 conv2d_59 (Conv2D)          (None, 48, 48, 32)        6944      
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 24, 24, 32)       0         
 g2D)                                                

In [39]:
filters = (8 * (2 * 0 + 1), 8 * (2 * 0 + 2))
filters, filters[-1]

((8, 16), 16)

In [40]:
last_input_size = [96,96,1]
pool_size = (2,2)

In [41]:
np.array(last_input_size).dtype

dtype('int64')

In [42]:
last_input_size = list((np.array(last_input_size[:2]) / pool_size[0]).astype("int64")) + [filters[-1]]
last_input_size

[48, 48, 16]